<h4 align="center">PLEASE OPEN THIS FILE IN JUPYTER, AS GITHUB CAN'T DISPLAY IT PROPERLY.</h4>

### Let's build an image classifier

### Dataset - MNIST

* large database of handwritten digits
* contains 60,000 training images and 10,000 testing images
* contains digits from 0 to 9
* consit of images of size 28x28 pixel
* http://yann.lecun.com/exdb/mnist/

<center><img src="img/mnist.jpeg" width="600" /></center>

### Fully-connected Neural Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

In [3]:
transformations = transforms.Compose([transforms.ToTensor()])

In [4]:
train_set = datasets.MNIST(root='./data', train=True, transform=transformations, download=True)
test_set  = datasets.MNIST(root='./data', train=False, transform=transformations, download=True)

In [5]:
batch_size = 100

In [6]:
train_set_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=False
)

In [7]:
test_set_loader = torch.utils.data.DataLoader(
    dataset=test_set,
    batch_size=batch_size,
    shuffle=False
)

In [8]:
class FullyConnectedClassifier(nn.Module):
    
    def __init__(self, input_size, output_size):
    
        super(FullyConnectedClassifier, self).__init__()

        self.fc1 = nn.Linear(input_size, 512)      # 1st hidden layer - 512 neurons
        self.fc2 = nn.Linear(512, 256)             # 2nd hidden layer - 256 neurons
        self.fc3 = nn.Linear(256, output_size)     # output layer     - 10 classes
        
    def forward(self, x):

        # flatten 2D grayscale image to a vector of 784 elements
        x = x.view(-1, 28*28)
        
        # 1st hidden layer (fully-connected)
        x = self.fc1(x)
        
        # apply non-linear transformation - sigmoid
        x = torch.sigmoid(x)

        # 2nd hidden layer (fully-connected)        
        x = self.fc2(x)
        
        # apply non-linear transformation - sigmoid
        x = torch.sigmoid(x)

        # final output layer (fully-connected)
        x = self.fc3(x)

        # return final result
        return x

In [9]:
model = FullyConnectedClassifier(28*28, 10)

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
n_epochs = 10

for epoch in range(1, n_epochs+1):

    # swith to train mode
    model = model.train()
    
    # global statistics
    running_train_loss = 0.
    
    for batch_idx, (inputs, targets) in enumerate(train_set_loader, start=1):

        # convert tensor to autograd.Variable
        inputs, targets = Variable(inputs), Variable(targets)

        # forward pass
        predictions = model.forward(inputs)

        # calculate loss - cross-entropy
        loss = criterion(predictions, targets)
        
        # accumulate loss
        running_train_loss += loss.item()

        # compute gradients
        loss.backward()
        
        # update weights based on calculated gradients
        optimizer.step()

        # clear all gradients after weights are updated
        optimizer.zero_grad()

    # print epoch statistics    
    epoch_train_loss = running_train_loss / len(train_set_loader.dataset)
    print('=> epoch: %d, train loss: %.6f' % (epoch, epoch_train_loss))
        
    # switch to evaluation/test mode
    model = model.eval()

    # global statistics
    running_test_loss = 0.
    running_test_corrects = 0.

    # disable calculation of gradients
    with torch.no_grad():

        for batch_idx, (inputs, targets) in enumerate(test_set_loader, start=1):

            # convert tensor to autograd.Variable
            inputs, targets = Variable(inputs), Variable(targets)
            
            # forward pass - make predictions
            predictions = model.forward(inputs)

            # calculate loss
            loss = criterion(predictions, targets)

            # convert probability to a predicted label
            _, prediction_labels = torch.max(predictions.data, 1)
            
            # calculate how many digits in the batch where predicted correctly
            running_test_corrects += torch.sum(prediction_labels == targets.data).item()

            # accumulate loss
            running_test_loss += loss.item()

    # print epoch statistics    
    epoch_test_loss = running_test_loss / len(test_set_loader.dataset)
    epoch_test_accuracy = running_test_corrects / len(test_set_loader.dataset)
    print('=> epoch: %d, test loss: %.6f, accuracy: %.3f \n' % (epoch, epoch_test_loss, epoch_test_accuracy))

=> epoch: 1, train loss: 0.022181
=> epoch: 1, test loss: 0.018247, accuracy: 0.477 

=> epoch: 2, train loss: 0.010925
=> epoch: 2, test loss: 0.007280, accuracy: 0.780 

=> epoch: 3, train loss: 0.005923
=> epoch: 3, test loss: 0.004924, accuracy: 0.850 

=> epoch: 4, train loss: 0.004551
=> epoch: 4, test loss: 0.004155, accuracy: 0.875 

=> epoch: 5, train loss: 0.004012
=> epoch: 5, test loss: 0.003778, accuracy: 0.889 

=> epoch: 6, train loss: 0.003717
=> epoch: 6, test loss: 0.003542, accuracy: 0.896 

=> epoch: 7, train loss: 0.003522
=> epoch: 7, test loss: 0.003371, accuracy: 0.900 

=> epoch: 8, train loss: 0.003372
=> epoch: 8, test loss: 0.003233, accuracy: 0.905 

=> epoch: 9, train loss: 0.003243
=> epoch: 9, test loss: 0.003112, accuracy: 0.909 

=> epoch: 10, train loss: 0.003126
=> epoch: 10, test loss: 0.003002, accuracy: 0.912 

